In [1]:
%matplotlib inline

In [3]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib as mpl
# mpl.use('Agg')
import seaborn as sns
from graph_tool.draw import sfdp_layout, graph_draw
from sklearn.metrics import average_precision_score

from helpers import infected_nodes
from graph_helpers import load_graph_by_name
from viz_helpers import lattice_node_pos

In [42]:
graph = 'p2p'
sampling_method = 'cut'
obs_method = "leaves"
dataset_id = '{}-mic-s0.02-o{}'.format(graph, obs_method)
cascade_dir = 'cascade/{}'.format(dataset_id)
query_path = 'outputs/queries-weighted/{}/{}/'.format(dataset_id, sampling_method)
inf_probas_path = 'outputs/inf_probas-weighted/{}/{}/'.format(dataset_id, sampling_method)
fig_root_dir = 'figs/infection-probas-after-queries/{}'.format(dataset_id)

In [43]:
def query_strategy_hit_ratio(method):
    queries_states = []
    for cascade_id in range(96):
        cascade_path = 'cascade-weighted/{}/{}.pkl'.format(dataset_id, cascade_id)
        obs, c = pkl.load(open(cascade_path, 'rb'))
        inf_nodes = set(infected_nodes(c))
        # inf_probas_list = pkl.load(open(inf_probas_path + method + '/{}.pkl'.format(cascade_id), 'rb'))
        queries, _ = pkl.load(open(query_path + method + '/{}.pkl'.format(cascade_id), 'rb'))
        queries_states += list(map(inf_nodes.__contains__, queries))
    queries_states = np.array(queries_states)
    
    return queries_states.sum() / len(queries_states)


In [44]:
methods = ['random', 'pagerank', 'entropy', 'prediction_error']
for m in methods:
    print('{}, hit ratio: {}'.format(m, query_strategy_hit_ratio(m)))

random, hit ratio: 0.00625
pagerank, hit ratio: 0.7114583333333333
entropy, hit ratio: 0.18645833333333334
prediction_error, hit ratio: 0.07291666666666667
